In [1]:
using Compat
using MomentumBasedControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot

import RigidBodyDynamics.Graphs: target

In [2]:
# load URDF
mechanism = AtlasRobot.mechanism()
remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [4]:
struct StandingController{M<:MomentumBasedController}
    lowlevel::M
    robotmass::Float64

    foottasks::Dict{RigidBody{Float64}, SpatialAccelerationTask}
    comtask::LinearMomentumRateTask
    pelvistask::AngularAccelerationTask
    jointtasks::Dict{JointID, JointAccelerationTask{Revolute{Float64}}}

    comgains::PDGains{Float64,Float64}
    pelvisgains::PDGains{Float64,Float64}
    jointgains::Dict{JointID, PDGains{Float64, Float64}}

    comref::Point3D{SVector{3, Float64}}
    jointrefs::Dict{JointID, Float64}
end

In [5]:
function StandingController(
        lowlevel::MomentumBasedController,
        feet::Vector{<:RigidBody},
        pelvis::RigidBody,
        nominalstate::MechanismState)
    mechanism = lowlevel.state.mechanism
    world = root_body(mechanism)
    m = mass(mechanism)
    
    # tasks
    foottasks = Dict(foot => SpatialAccelerationTask(mechanism, path(mechanism, world, foot)) for foot in feet)
    addtask!.(lowlevel, collect(values(foottasks)))
    
    comtask = LinearMomentumRateTask(mechanism, centroidal_frame(lowlevel))
    addtask!(lowlevel, comtask)
    
    pelvistask = AngularAccelerationTask(mechanism, path(mechanism, world, pelvis))
    addtask!(lowlevel, pelvistask)

    revolutejoints = filter(j -> joint_type(j) isa Revolute, tree_joints(mechanism))
    legjoints = vcat(collect(task.path) for task in foottasks)
    positioncontroljoints = setdiff(revolutejoints, legjoints)
    jointtasks = Dict(JointID(j) => JointAccelerationTask(j) for j in positioncontroljoints)
    addtask!.(lowlevel, collect(values(jointtasks)))
    
    # gains
    comgains = PDGains(10., 3.)
    pelvisgains = PDGains(10., 2.)
    jointgains = Dict(jointid => PDGains(100.0, 20.) for jointid in keys(jointtasks))
    
    # references
    comref = center_of_mass(nominalstate)
    jointrefs = Dict(jointid => configuration(nominalstate, jointid)[1] for jointid in keys(jointtasks))
    
    StandingController(
        lowlevel, m,
        foottasks, comtask, pelvistask, jointtasks,
        comgains, pelvisgains, jointgains,
        comref, jointrefs)
end

StandingController

In [14]:
function (controller::StandingController)(τ::AbstractVector, t::Number, state::MechanismState)
    # Linear momentum control
    com = center_of_mass(state)
    centroidal = centroidal_frame(controller.lowlevel)
    centroidal_to_world = Transform3D(centroidal, com.frame, com.v)
    world_to_centroidal = inv(centroidal_to_world)
    e = FreeVector3D(centroidal, transform(controller.comref, world_to_centroidal).v)
    ė = FreeVector3D(centroidal, linear(transform(momentum(state), world_to_centroidal)) / controller.robotmass)
    setdesired!(controller.comtask, pd(controller.comgains, e, ė))
    
    # Pelvis orientation control
    pelvistask = controller.pelvistask
    pelvisgains = controller.pelvisgains
    pelvis = target(pelvistask.path)
    Hpelvis = transform_to_root(state, pelvis)
    Tpelvis = transform(twist_wrt_world(state, pelvis), inv(Hpelvis))
    ωdesired = FreeVector3D(Tpelvis.frame, pd(pelvisgains, rotation(Hpelvis), Tpelvis.angular))
    setdesired!(pelvistask, ωdesired)
    
    # Joint position control
    for jointid in keys(controller.jointtasks)
        task = controller.jointtasks[jointid]
        gains = controller.jointgains[jointid]
        ref = controller.jointrefs[jointid]
        v̇desired = pd(gains, configuration(state, jointid)[1], ref, velocity(state, jointid)[1], 0.0)
        setdesired!(task, v̇desired)
    end
    
    controller.lowlevel(τ, t, state)
end

In [7]:
# create optimizer
using OSQP.MathOptInterfaceOSQP
MOI = MathOptInterface
optimizer = OSQPOptimizer()
MOI.set!(optimizer, OSQPSettings.Verbose(), false)
MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-8)
MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-16)
MOI.set!(optimizer, OSQPSettings.MaxIter(), 10000)
MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior

In [8]:
# create low level controller
num_basis_vectors = 4
lowlevel = MomentumBasedController{num_basis_vectors}(mechanism, optimizer);
for body in bodies(mechanism)
    for point in RigidBodyDynamics.contact_points(body)
        position = location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(lowlevel, body, position, normal, μ)
        contact.maxnormalforce = 1e9 # TODO
    end
end

In [9]:
# state initialization
function initialize!(state::MechanismState)
    mechanism = state.mechanism
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(mechanism, "$(sideprefix)_leg_kny")
        hippitch = findjoint(mechanism, "$(sideprefix)_leg_hpy")
        anklepitch = findjoint(mechanism, "$(sideprefix)_leg_aky")
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        set_configuration!(state, anklepitch, [-kneebend / 2 - hipbendextra])
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 0.85])
    state
end

initialize! (generic function with 1 method)

In [10]:
# create standing controller
feet = findbody.(mechanism, ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
initialize!(nominalstate)
controller = StandingController(lowlevel, feet, pelvis, nominalstate);

In [17]:
state = MechanismState(mechanism)
initialize!(state)
τ = similar(velocity(state))
controller(τ, 0.0, state)

In [18]:
# set up visualizer
using MeshCat
using MeshCatMechanisms
vis = Visualizer()[:atlas]
visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
mvis = MechanismVisualizer(mechanism, visuals, vis)
set_configuration!(mvis, configuration(nominalstate))
IJuliaCell(mvis)

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/08b76a9f0aa6406b9b54525b8451334ac21dd778-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/08b76a9f0aa6406b9b54525b8451334ac21dd778-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


WebIO.Scope("scope-9c7a90ce-bfb3-4e9c-bc84-70555902a814", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("height", "100%"),Pair{String,Any}("scrolling", "no"),Pair{String,Any}("src", "javascript:void(0)"),Pair{String,Any}("frameborder", 0))),Pair{Symbol,Any}(:id, "ifr"),Pair{Symbol,Any}(:style, Dict("width"=>"100%","flexGrow"=>"1"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:style, Dict("flexDirection"=>"column","height"=>"100%","minHeight"=>"400px","display"=>"flex","width"=>"100%","overflow"=>"hidden"))), 1), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(), Set{String}(), nothing, Any[], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function () {\n    var frame = this.dom.querySelector(\"#ifr\");\n    var doc = frame.contentDocument\n    var win = frame.contentWindow\n\n    // Determine if we're running on a Jupyter hosting service\n    // that requires a base URL when retrieving assets\n    var curMatch =\n        window.location.href\n        .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n    curMatch = curMatch ||\n        window.location.href\n        .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n    if (curMatch) {\n        var base = doc.createElement(\"base\");\n        base.setAttribute(\"href\", curMatch[1] + '/');\n        doc.head.appendChild(base);\n    }\n\n    var webio = doc.createElement(\"script\")\n    webio.src = \"pkg/WebIO/webio/dist/bundle.js\"\n    var parent = window\n\n    function resizeIframe() {\n        doc.body.style.padding = '0'\n        doc.body.style.margin = '0'\n        doc.documentElement.height = '100%'\n        doc.body.height = '100%'\n    }\n\n    webio.onload = function () {\n        win.WebIO.sendCallback = parent.WebIO.sendCallback; // Share stuff\n        win.WebIO.scopes = parent.WebIO.scopes\n        win.WebIO.obsscopes = parent.WebIO.obsscopes\n        win.WebIO._connected = true\n        doc.body.innerHTML = \"<html><body>\" + \"<div class='display:none'></div><unsafe-script style='display:none'>\\nWebIO.mount(this.previousSibling,{&quot;props&quot;:{},&quot;nodeType&quot;:&quot;Scope&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;imports&quot;:{&quot;data&quot;:[{&quot;name&quot;:null,&quot;type&quot;:&quot;js&quot;,&quot;url&quot;:&quot;pkg/MeshCat/meshcat/dist/main.min.js&quot;}],&quot;type&quot;:&quot;async_block&quot;},&quot;id&quot;:&quot;scope-f8dd2001-a485-4bda-b917-c5d5b44ec4bc&quot;,&quot;handlers&quot;:{&quot;meshcat-command&quot;:[(function (val){return this.viewer.handle_command_message({&quot;data&quot;:val})})],&quot;_promises&quot;:{&quot;importsLoaded&quot;:[(function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({&quot;name&quot;:&quot;meshcat-request&quot;,&quot;scope&quot;:&quot;scope-f8dd2001-a485-4bda-b917-c5d5b44ec4bc&quot;,&quot;id&quot;:&quot;ob_02&quot;,&quot;type&quot;:&quot;observable&quot;},String(Date.now())); return window.document.body.style.margin=&quot;0&quot;})]}},&quot;systemjs_options&quot;:null,&quot;observables&quot;:{&quot;meshcat-command&quot;:{&quot;sync&quot;:false,&quot;id&quot;:&quot;ob_01&quot;,&quot;value&quot;:[131,166,109,97,116,114,105,120,199,64,23,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,164,116,121,112,101,173,115,101,116,95,116,114,97,110,115,102,111,114,109,164,112,97,116,104,218,0,99,47,109,101,115,104,99,97,116,47,97,116,108,97,115,47,119,111,114,108,100,47,112,101,108,118,105,115,47,108,116,111,114,115,111,47,109,116,111,114,115,111,47,117,116,111,114,115,111,47,114,95,99,108,97,118,47,114,95,115,99,97,112,47,114,95,117,97,114,109,47,114,95,108,97,114,109,47,114,95,117,102,97,114,109,47,114,95,108,102,97,114,109,47,114,95,104,97,110,100]},&quot;meshcat-contro

In [20]:
# simulate
using RigidBodySim
state = MechanismState(mechanism)
initialize!(state)
Δt = 0.003
dynamics = Dynamics(mechanism, PeriodicController(similar(velocity(state)), Δt, controller))
vis_callbacks = CallbackSet(mvis, state);
problem = ODEProblem(dynamics, state, (0., 1.), callback = vis_callbacks);

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 1.0)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
@time sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = vis_callbacks)